In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-tdxjfxqj/unsloth_6f6d338a43334fc3b194f585c0e7111e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-tdxjfxqj/unsloth_6f6d338a43334fc3b194f585c0e7111e
  Resolved https://github.com/unslothai/unsloth.git to commit c45a0c3e10f1fd487921de285fd33b4cd38f64ce
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 10.3 MB/s eta 0:00:00
   ━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.12.2
    Uninstalling trl-0.12.2:
      Successfully uninstalled trl-0.12.2


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/Qwen2.5-1.5B-bnb-4bit",
]
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-1.5B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.2: Fast Qwen2 patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA RTX A4000. Max memory: 15.729 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.12.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Input:
{}
### Response:
{}"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [ ]:
# from datasets import load_dataset
# dataset = load_dataset("raiyan007/merged_dataset_v1", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("raiyan007/modified_camel-ai_math", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)
# from datasets import load_dataset
# dataset = load_dataset("raiyan007/modified_camel-ai_math", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
!pip install tensorboard


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 20,
        max_steps = 3250,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        logging_dir="logs",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        evaluation_strategy="no",  # Evaluate every few steps
        eval_steps=50,  # Evaluate every 50 steps
        save_steps=50,  # Save model every 50 steps
        save_total_limit=2,  # Keep only the last two checkpoints
        report_to="tensorboard",  # Log metrics to TensorBoard
    ),
)

/home/intisar/miniconda3/envs/llm-495B-py310/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/50000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
!tensorboard --logdir=logs


TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.18.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 3,250
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
1,0.598200
2,0.729100
3,0.657600
4,0.653200
5,0.656700
6,0.670200
7,0.729000
8,0.785000
9,0.686000
10,0.637400


In [ ]:
model.save_pretrained("finetuned/qwen/math/Qwen2.5-1.5B-bnb-4bit") # Local saving
tokenizer.save_pretrained("finetuned/qwen/math/Qwen2.5-1.5B-bnb-4bit")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('finetuned/qwen/math/Qwen2.5-1.5B-bnb-4bit/tokenizer_config.json',
 'finetuned/qwen/math/Qwen2.5-1.5B-bnb-4bit/special_tokens_map.json',
 'finetuned/qwen/math/Qwen2.5-1.5B-bnb-4bit/vocab.json',
 'finetuned/qwen/math/Qwen2.5-1.5B-bnb-4bit/merges.txt',
 'finetuned/qwen/math/Qwen2.5-1.5B-bnb-4bit/added_tokens.json',
 'finetuned/qwen/math/Qwen2.5-1.5B-bnb-4bit/tokenizer.json')

In [ ]:

from huggingface_hub import HfApi, login

# Login to Hugging Face (you'll need to do this interactively)


# Push the model
model.push_to_hub("raiyan007/Qwen2.5-1.5B-bnb-4bit_math")
tokenizer.push_to_hub("raiyan007/Qwen2.5-1.5B-bnb-4bit_math")

README.md:   0%|          | 0.00/580 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/73.9M [00:00<?, ?B/s]

Saved model to https://huggingface.co/raiyan007/Qwen2.5-1.5B-bnb-4bit_math


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
# Make sure model is correctly unpacked from the returned tuple
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="./finetuned/qwen/math/Qwen2.5-1.5B-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Now use the model
FastLanguageModel.for_inference(model)  # Enable faster inference
inputs = tokenizer(
    [
        prompt.format(
            "Given a concept of Algebra. Solving linear equations",  # instruction
            "Solve for x: 3x + 5 = 17.",  # input
            "",  # output
        )
    ],
    return_tensors="pt"
).to("cuda")

# Generate text
# output_ids = model.generate(**inputs, max_new_tokens=128)
# generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# # Print the generated text
# print(generated_text)
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


# Print the generated text


==((====))==  Unsloth 2024.12.2: Fast Qwen2 patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA RTX A4000. Max memory: 15.729 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Given a concept of Algebra. Solving linear equations
### Input:
Solve for x: 3x + 5 = 17.
### Response:
To solve for x, we need to isolate x on one side of the equation. We can do this by following these steps:

1. Subtract 5 from both sides of the equation:
3x + 5 - 5 = 17 - 5
3x = 12

2. Divide both sides of the equation by 3:
3x / 3 = 12 / 3
x = 4

So, the so

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

In [ ]:
#eval

In [ ]:
def benchmark_dataset_metrics(dataset):
    print("Dataset Metrics:")
    print(f"Total Samples: {len(dataset)}")
    print(f"Sample Instruction Examples:")
    for i in range(min(3, len(dataset))):
        print(f"Instruction {i+1}: {dataset[i]['instruction']}")

    # Calculate average lengths
    instruction_lengths = [len(item['instruction']) for item in dataset]
    input_lengths = [len(item['input']) for item in dataset]
    output_lengths = [len(item['output']) for item in dataset]

    print("\nLength Statistics:")
    print(f"Avg Instruction Length: {sum(instruction_lengths) / len(instruction_lengths):.2f}")
    print(f"Avg Input Length: {sum(input_lengths) / len(input_lengths):.2f}")
    print(f"Avg Output Length: {sum(output_lengths) / len(output_lengths):.2f}")

# Call the function
benchmark_dataset_metrics(dataset)

Dataset Metrics:
Total Samples: 51760
Sample Instruction Examples:
Instruction 1: Give three tips for staying healthy.
Instruction 2: What are the three primary colors?
Instruction 3: Describe the structure of an atom.

Length Statistics:
Avg Instruction Length: 62.41
Avg Input Length: 24.34
Avg Output Length: 678.94


In [ ]:
!pip install scikit-learn

In [ ]:
!pip install deepeval

In [ ]:
import torch
import re
import random
import logging
# Define parameters
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load the previously fine-tuned model as the base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "finetuned/qwen/alpaca_math/Qwen2.5-1.5B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Input:
{}
### Response:
{}"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

# Set up logging configuration
logging.basicConfig(
    filename='./finetuned/model_benchmark.log',  # You can change this to your desired log file
    level=logging.INFO,              # Log messages with level INFO and above
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def evaluate_generation_quality(model, tokenizer, dataset, num_samples=300):
    model.eval()

    # Randomly sample from the dataset
    sampled_indices = random.sample(range(len(dataset)), min(num_samples, len(dataset)))
    results = []
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for idx in sampled_indices:
            # Prepare input
            input_text = prompt.format(
                dataset[idx]['instruction'],
                dataset[idx]['input'],
                ""  # Empty response to generate
            )

            # Tokenize input
            inputs = tokenizer(input_text, return_tensors='pt').to('cuda')

            # Generate response
            outputs = model.generate(
                **inputs,
                max_new_tokens=128,  # Adjust as needed
                do_sample=True,
                temperature=0.7
            )

            # Decode generated text
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Check if the generated output matches the actual output
            if generated_text.strip() == dataset[idx]['output'].strip():
                correct_predictions += 1
            total_predictions += 1

            # Compare generated vs actual
            results.append({
                'instruction': dataset[idx]['instruction'],
                'input': dataset[idx]['input'],
                'generated_output': generated_text,
                'actual_output': dataset[idx]['output']
            })

    # Calculate accuracy
    accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0
    logging.info(f"Accuracy: {accuracy:.2f}% for {num_samples} samples")

    # Print results to console
    print("\n--- Generation Results ---")
    for result in results[:5]:  # Print first 5 results for brevity
        print(f"Instruction: {result['instruction']}")
        print(f"Input: {result['input']}")
        print(f"Generated Output: {result['generated_output']}")
        print(f"Actual Output: {result['actual_output']}")
        print("---")

    return results, accuracy

def calculate_perplexity(model, tokenizer, dataset):
    model.eval()
    total_loss = 0
    total_tokens = 0

    # Randomly sample for perplexity calculation
    sampled_indices = random.sample(range(len(dataset)), min(100, len(dataset)))

    with torch.no_grad():
        for idx in sampled_indices:
            # Prepare full text
            full_text = prompt.format(
                dataset[idx]['instruction'],
                dataset[idx]['input'],
                dataset[idx]['output']
            ) + tokenizer.eos_token

            # Tokenize
            inputs = tokenizer(full_text, return_tensors='pt').to('cuda')

            # Forward pass
            outputs = model(**inputs, labels=inputs['input_ids'])

            # Accumulate loss
            total_loss += outputs.loss.item() * inputs['input_ids'].numel()
            total_tokens += inputs['input_ids'].numel()

    # Calculate perplexity
    perplexity = torch.exp(torch.tensor(total_loss / total_tokens))
    print(f"\nDataset Perplexity (on random sample): {perplexity.item()}")
    return perplexity.item()

def comprehensive_benchmark(model, tokenizer, dataset, num_samples=300):
    # Randomly sample instruction domains
    unique_instructions = list(set(dataset['instruction']))
    sampled_instruction_domains = random.sample(unique_instructions, min(5, len(unique_instructions)))

    benchmark_results = {
        'dataset_metrics': {
            'total_samples': len(dataset),
            'sampled_instruction_domains': sampled_instruction_domains
        },
        'generation_quality': evaluate_generation_quality(model, tokenizer, dataset, num_samples),
        'perplexity': calculate_perplexity(model, tokenizer, dataset)
    }

    # Print the benchmark results
    print("\n--- Benchmark Results ---")
    print(f"Total Samples: {benchmark_results['dataset_metrics']['total_samples']}")
    print(f"Sampled Instruction Domains: {benchmark_results['dataset_metrics']['sampled_instruction_domains']}")
    print(f"Perplexity: {benchmark_results['perplexity']}")

    return benchmark_results

# Run the comprehensive benchmark with random sampling
comprehensive_benchmark_results = comprehensive_benchmark(model, tokenizer, dataset, num_samples=300)


==((====))==  Unsloth 2024.12.2: Fast Qwen2 patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA RTX A4000. Max memory: 15.729 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

--- Generation Results ---
Instruction: Identify four elements of a successful blog post.
Input: 
Generated Output: Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Identify four elements of a successful blog post.
### Input:

### Response:
1. Clear and concise title: A strong and engaging title is essential to grab the reader's attention and get them to read the rest of the post. It should be descriptive and

In [ ]:
from unsloth import FastLanguageModel
import torch

# Define parameters
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load the previously fine-tuned model as the base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "finetuned/qwen/math/Qwen2.5-1.5B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Apply PEFT (parameter-efficient fine-tuning)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# Define the prompt formatting function
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Input:
{}
### Response:
{}"""
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Load and preprocess the dataset
from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

# Define training arguments and start fine-tuning
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 20,
        max_steps = 3250,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        logging_dir="logs",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        evaluation_strategy="no",
        save_steps=50,
        save_total_limit=2,
        report_to="tensorboard",
    ),
)

# Start training
trainer_stats = trainer.train()

# Save the fine-tuned model locally
model.save_pretrained("finetuned/qwen/alpaca_math/Qwen2.5-1.5B-bnb-4bit")
tokenizer.save_pretrained("finetuned/qwen/alpaca_math/Qwen2.5-1.5B-bnb-4bit")

# Push the model to Hugging Face Hub
from huggingface_hub import HfApi

# Login and push the model (interactive login may be required)
model.push_to_hub("raiyan007/Qwen2.5-1.5B-bnb-4bit_alpaca_math")
tokenizer.push_to_hub("raiyan007/Qwen2.5-1.5B-bnb-4bit_alpaca_math")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.2: Fast Qwen2 patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA RTX A4000. Max memory: 15.729 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.12.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.
/home/intisar/miniconda3/envs/llm-495B-py310/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 3,250
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
1,1.613700
2,1.696100
3,1.676800
4,1.515200
5,1.684300
6,1.332900
7,1.438800
8,1.231300
9,1.450900
10,1.170500


...

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/73.9M [00:00<?, ?B/s]

Saved model to https://huggingface.co/raiyan007/Qwen2.5-1.5B-bnb-4bit_alpaca_math


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
from unsloth import FastLanguageModel
import torch

# Define parameters
max_seq_length = 2048
dtype = None
load_in_4bit = True
# Make sure model is correctly unpacked from the returned tuple
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="raiyan007/Qwen2.5-1.5B-bnb-4bit_math",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Input:
{}
### Response:
{}"""
# Now use the model
FastLanguageModel.for_inference(model)  # Enable faster inference
inputs = tokenizer(
    [
        prompt.format(
            "Solve a math problem in Algebra. Solving linear equations",  # instruction
            "Solve the equation: 3x + 5 = 14.",  # input
            "",  # output
        )
    ],
    return_tensors="pt"
).to("cuda")

# Generate text
# output_ids = model.generate(**inputs, max_new_tokens=128)
# generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# # Print the generated text
# print(generated_text)
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


# Print the generated text


==((====))==  Unsloth 2024.12.4: Fast Qwen2 patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Solve a math problem in Algebra. Solving linear equations
### Input:
Solve the equation: 3x + 5 = 14.
### Response:
To solve the equation 3x + 5 = 14, we will follow these steps:

Step 1: Subtract 5 from both sides of the equation.
3x + 5 - 5 = 14 - 5
3x = 9

Step 2: Divide both sides of the equation by 3.
3x / 3 = 9 / 3
x = 3

So, the solution to the equation 3x + 5 =

In [ ]:
!pip install git+https://github.com/EleutherAI/lm-evaluation-harness
!pip install bitsandbytes

  Cloning https://github.com/EleutherAI/lm-evaluation-harness to /tmp/pip-req-build-v_uue2xl
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/lm-evaluation-harness /tmp/pip-req-build-v_uue2xl
  Resolved https://github.com/EleutherAI/lm-evaluation-harness to commit bcb4cbf464668450f1969a4f1bef28f8638d1648
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 22.9 MB/s eta 0:00:00

In [ ]:
from huggingface_hub import notebook_login


In [ ]:
notebook_login()


In [ ]:
!lm_eval --model hf \
    --model_args pretrained=raiyan007/Qwen2.5-1.5B-bnb-4bit_alpaca_math,dtype="float16" \
    --tasks truthfulqa,hellaswag \
    --device cuda:0 \
    --batch_size 6\
    --output_path ./results \
    --log_samples

2024-12-07 21:50:37.428479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 21:50:37.448820: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 21:50:37.455213: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 21:50:37.469286: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-07 21:50:38.585739: W tensorflow/comp